In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier

from collections import Counter
import pandas as pd
import math

In [2]:
def fit_and_predict(X, Y, modelo, porcentagem_treino=0.9, porcentagem_teste=0.1):
    """Agrupa o processo de segregar os dados, treinar o modelo e emitir a taxa de acerto
    
    Args:
        X (dataframe): conjunto de valores para as features do modelo
        Y (dataframe): valores da função alvo para o input X (f(X) = Y)
        modelo (scikit model): modelo da biblioteca sklearn
        porcentagem_treino (float): decimal do conjunto de dados para treino
        porcentagem_teste (float): decimal do conjunto de dados para teste
        
    Returns:
        taxa_de_acerto(float): taxa de acerto relativa do modelo em decimal
    """
    tamanho_de_treino = math.floor(porcentagem_treino * len(Y))
    tamanho_de_teste = tamanho_de_treino + math.floor(porcentagem_teste*len(Y))
    
    # Divisão do conjunto de dados
    treino_dados = X[:tamanho_de_treino]
    treino_marcacoes = Y[:tamanho_de_treino]
    
    teste_dados = X[tamanho_de_treino:(tamanho_de_treino+tamanho_de_teste)]
    teste_marcacoes = Y[tamanho_de_treino:(tamanho_de_treino+tamanho_de_teste)]
    
    validacao_dados = X[(tamanho_de_treino+tamanho_de_teste):]
    validacao_marcacoes = Y[(tamanho_de_treino+tamanho_de_teste):]
    
    # Execução do modelo
    modelo.fit(treino_dados, treino_marcacoes)

    resultado = modelo.predict(teste_dados)
    taxa_de_acerto = sum(resultado==teste_marcacoes)/len(teste_marcacoes)

    return taxa_de_acerto

# 01. Classificando E-mails, animais e muito mais

In [3]:
# Introdução para exemplificar no que consiste a classificação (parecido com o do curso de sklearn)
porco1 = [1, 1, 0]
porco2 = [1, 1, 0]
porco3 = [1, 1, 0]
cachorro4 = [1, 1, 1]
cachorro5 = [0, 1, 1]
cachorro6 = [0, 1, 1]

dados = [porco1, porco2, porco3, cachorro4, cachorro5, cachorro6]

# 1 -> porco, -1 -> cachorro
marcacoes = [1, 1, 1, -1, -1, -1]

modelo = MultinomialNB()
modelo.fit(dados, marcacoes)

misterioso1 = [1, 1, 1]
misterioso2 = [1, 0, 0]
misterioso3 = [0, 0, 1]
marcacoes_teste = [-1, 1, 1]
teste = [misterioso1, misterioso2, misterioso3]

resultado = modelo.predict(teste)
taxa_de_acerto = sum(resultado==marcacoes_teste)/len(marcacoes_teste)

print(f'Taxa de acerto de {round(taxa_de_acerto*100, 2)}%.')

Taxa de acerto de 66.67%.


# 02. Importando, classificando e validando um modelo

In [18]:
dados = pd.read_csv('dados/acesso.csv')
dados.head()

X = dados[['home', 'como_funciona', 'contato']]
Y = dados['comprou']

treino_dados = X[:90]
treino_marcacoes = Y[:90]

teste_dados = X[90:]
teste_marcacoes = Y[90:]

modelo = MultinomialNB()
modelo.fit(treino_dados, treino_marcacoes)

resultado = modelo.predict(teste_dados)

taxa_de_acerto = sum(resultado==teste_marcacoes)/len(teste_marcacoes)
print(f'Taxa de acerto de {round(taxa_de_acerto*100, 2)}%.')

Taxa de acerto de 88.89%.


# 03. Classificação de variáveis categóricas

In [19]:
df = pd.read_csv('dados/busca.csv')
df.head()

,home,busca,logado,comprou
0,0,algoritmos,1,1
1,0,java,0,1
2,1,algoritmos,0,1
3,1,ruby,1,0
4,1,ruby,0,1


In [6]:
X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']

X = pd.get_dummies(X_df).astype(int)
Y = Y_df

taxa_de_acerto = fit_and_predict(X, Y, modelo=MultinomialNB())
print(f'Taxa de acerto de {round(taxa_de_acerto*100, 2)}%.')

Taxa de acerto de 82.0%.


# 04. O problema do sucesso e o algoritmo burro

In [20]:
df = pd.read_csv('dados/busca.csv')
df.head()

X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']

X = pd.get_dummies(X_df).astype(int)
Y = Y_df

In [8]:
# Baseline
acerto_base = max(Counter(teste_marcacoes).values()) #No Python3, utilizamos o método values(). 
taxa_de_acerto_base = acerto_base / len(teste_marcacoes)

# Algoritmo
taxa_de_acerto_algoritmo = fit_and_predict(X, Y, modelo=MultinomialNB())

print(f'Taxa de acerto do algoritmo: {round(taxa_de_acerto_algoritmo*100, 2)}%.')
print(f'Taxa de acerto base: {round(taxa_de_acerto_base*100, 2)}%.')

Taxa de acerto do algoritmo: 82.0%.
Taxa de acerto base: 77.78%.


# 06. Testando diferentes modelos e validando o vencedor

In [21]:
df = pd.read_csv('dados/busca.csv')
df.head()

X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']

X = pd.get_dummies(X_df).astype(int)
Y = Y_df

porcentagem_treino = 0.8
porcentagem_teste = 0.1 # Padrão

# Para armazenar as instâncias de cada modelo e guardar o fit
modelo_nb = MultinomialNB()
modelo_ada = AdaBoostClassifier()

taxa_de_acerto_NB = fit_and_predict(X, Y, modelo=modelo_nb, porcentagem_treino=porcentagem_treino)
taxa_de_acerto_adaboost = fit_and_predict(X, Y, modelo=A, 
                                          porcentagem_treino=porcentagem_treino)
taxa_de_acerto_base = max(Counter(teste_marcacoes).values())/len(teste_marcacoes)

print(f'Taxa de acerto MultinomialNB: {round(taxa_de_acerto_NB*100, 2)}%.')
print(f'Taxa de acerto AdaBoost: {round(taxa_de_acerto_adaboost*100, 2)}%.')
print(f'Taxa de acerto base: {round(taxa_de_acerto_base*100, 2)}%.')
print('\n')
print(f'Qtd. utilizada para testes: {math.floor(0.1*len(Y))}.')

Taxa de acerto MultinomialNB: 82.0%.
Taxa de acerto AdaBoost: 84.5%.
Taxa de acerto base: 77.78%.


Qtd. utilizada para testes: 100.


In [17]:
# Validação
tamanho_validacao = math.floor(porcentagem_treino*len(Y) + math.floor(porcentagem_teste*len(Y)))

validacao_dados = X[tamanho_validacao:]
validacao_marcacoes = Y[tamanho_validacao:]
                         
if taxa_de_acerto_NB > taxa_de_acerto_adaboost:
    modelo_vencedor = MultinomialNB()
else:
    modelo_vencedor = A

taxa_de_acerto_validacao = sum(modelo_vencedor.predict(validacao_dados) == validacao_marcacoes)/len(validacao_marcacoes)
print(f'Taxa de acerto de validação para o {type(modelo_vencedor).__name__}: {round(taxa_de_acerto_validacao*100, 2)}%.')
print(f'Qtd. utilizada para validação: {len(validacao_marcacoes)}.')

Taxa de acerto de validação para o AdaBoostClassifier: 85.0%.
Qtd. utilizada para validação: 100.
